# ENN583 - Week 2 - Practical

In this week's Prac we will be explore OpenCV's functions for interest points detection, description, and matching.


We will be working with one of the Kitti dataset sequences, the same we used last week. If you have not already downloaded it, go back to last week's Prac.


## Load the KITTI Sequence

In [ ]:
# Check if pykitti is installed, if not use pip to set it up
try:
    import pykitti
except:
    !pip install pykitti
    import pykitti

# Read the dataset sequence we just downloaded
basedir = '../kitti'
date = '2011_09_26'
drive = '0035'

# The 'frames' argument is optional - default: None, which loads the whole dataset.
# data = pykitti.raw(basedir, date, drive, frames=range(0, 50, 5))
data = pykitti.raw(basedir, date, drive)

## Open and Display Images from the Stereo Camera of the KITTI Sequence

Here is a reminder how we can get access to the left and right images from the colour stereo camera through the dataset class.

See last week's Prac notebook for a reminder of the different types of sensor data and ground truth pose that is stored in the dataset.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

# get the left and right stereo images from the 10th frame
left, right = data.get_rgb(10)

# use plt to show them side by side
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(left)
plt.subplot(1,2,2)
plt.imshow(right)
plt.show()


## Extract FAST Features

As a first demo, let us detect FAST corners in both images.


In [ ]:
# get the left and right image of the 10th stereo pair in this sequence and convert them to greyscale.
left, right = data.get_rgb(10)
left = cv2.cvtColor(np.array(left), cv2.COLOR_RGB2GRAY)  
right = cv2.cvtColor(np.array(right), cv2.COLOR_RGB2GRAY)

# detect FAST corners in both images
fast = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
kp1 = fast.detect(left, None)
kp2 = fast.detect(right, None)

# kp1 and kp2 are lists of keypoints, each of the class cv2.KeyPoint
# see the documentation at https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html for all the class members

# print the number of keypoints detected in each image
print(f'Left: {len(kp1)} keypoints')
print(f'Right: {len(kp2)} keypoints')

# let us inspect the interesting members of a keypoint (i.e. the ones that are not starting with an underscore)
print(f'Interesting Keypoint class members: {[m for m in dir(kp1[0]) if not m.startswith("_")]}')

# to get access to all of the keypoint coordinates, we can access their individual .pt members like this:
pt1 = np.array([x.pt for x in kp1])
pt2 = np.array([x.pt for x in kp2])

# we can manually draw the keypoints into the images
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.plot(pt1[:,0], pt1[:,1], 'r.', markersize=1)
plt.title('Left Image')
plt.imshow(left, cmap='gray')

plt.subplot(2,1,2)
plt.imshow(right, cmap='gray')
plt.plot(pt2[:,0], pt2[:,1], 'r.', markersize=1)
plt.title('Right Image')
plt.suptitle('Keypoints detected with FAST')
plt.show()


# for a quicker way to display keypoints, we can use the drawKeypoints function of OpenCV
left_corners = cv2.drawKeypoints(left, kp1, None, color=(255,0,0))
right_corners = cv2.drawKeypoints(right, kp2, None, color=(255,0,0))

# display the images
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.imshow(left_corners)
plt.title('Left Image')
plt.subplot(212)
plt.imshow(right_corners)
plt.title('Right Image')
plt.show()



## Try Different Interest Point Detectors

Let us now work with different detectors. 

Modify the code below and extract the following features:
 - SIFT (see https://docs.opencv.org/4.7.0/da/df5/tutorial_py_sift_intro.html and https://docs.opencv.org/4.7.0/d7/d60/classcv_1_1SIFT.html)
 - ORB (see https://docs.opencv.org/4.7.0/d1/d89/tutorial_py_orb.html and https://docs.opencv.org/4.7.0/db/d95/classcv_1_1ORB.html)

Display them and compare. 

In [ ]:
# get the left and right image of the 10th stereo pair in this sequence and convert them to greyscale.
left, right = data.get_rgb(10)
left = cv2.cvtColor(np.array(left), cv2.COLOR_RGB2GRAY)  
right = cv2.cvtColor(np.array(right), cv2.COLOR_RGB2GRAY)

# detect SIFT features in both images
sift = cv2.xfeatures2d.SIFT_create()       # Your turn! Explore different parameters you can pass into the SIFT constructor
kp1 = sift.detect(left, None)
kp2 = sift.detect(right, None)

# visualise the keypoints, including their scale and orientation
img = cv2.drawKeypoints(left, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.title('Left Image')
plt.imshow(img, cmap='gray')

img = cv2.drawKeypoints(right, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.subplot(2,1,2)
plt.imshow(img)
plt.title('Right Image')
plt.suptitle('Keypoints detected with SIFT')
plt.show()

# ================================ #
# Your turn! Change the feature detector to ORB and see what happens

orb = cv2.ORB_create()               # Your turn! Explore different parameters you can pass into the SIFT constructor

# YOUR TURN! CONTINUE YOUR CODE HERE



## Explore Feature Matching

Let's calculate some descriptors and try matching features between different images.

In [ ]:
# get the left and right image of the 10th stereo pair in this sequence and convert them to greyscale.
left, right = data.get_rgb(10)
left = cv2.cvtColor(np.array(left), cv2.COLOR_RGB2GRAY)  
right = cv2.cvtColor(np.array(right), cv2.COLOR_RGB2GRAY)

# detect SIFT features in both images
# this time we also calculate their descriptors
sift = cv2.xfeatures2d.SIFT_create()  
kp1, desc1 = sift.detectAndCompute(left, None)     
kp2, desc2 = sift.detectAndCompute(right, None)

# the simplest matching method is the Brute Force Matcher, cv2.BFMatcher (https://docs.opencv.org/4.7.0/d3/da1/classcv_1_1BFMatcher.html)
# create the BFMatcher object. Notice how we use the crossCheck flag to enable the 
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# match the descriptors from both images
matches = bf.match(desc1, desc2)

# matches is of type DMatch
print(f'There are {len(matches)} matches. They are of type {type(matches[0])}')
print(f'Each match has the following attributes: {[a for a in dir(matches[0]) if not a.startswith("_")]}')

# there is a convenient function that lets us display the matches
# we can specify the number of matches we want to display
# here we display the strongest 50 matches
# notice that the matches are sorted by distance
matches = sorted(matches, key = lambda x:x.distance)    # quickly sort the matches based on distance, smallest distance first
img = cv2.drawMatches(left,kp1,right,kp2,matches[:50], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(40,20))
plt.imshow(img)
plt.show()


# YOUR TURN! Plot a histogram of the distances of the matches. What do you notice?


# ... your code here ...


# Your turn!
# Now calculate ORB features and descriptors again, and match them using the Brute Force Matcher
# Pay attention to the distance metric we use for the matcher. Is the L2 Norm still the best choice?



## Manually Extract BRIEF Descriptors

We can also extract BRIEF descriptors around any location in an image. Let's try this!


In [ ]:
# get the left and right image of the 10th stereo pair in this sequence and convert them to greyscale.
left, right = data.get_rgb(10)
left = cv2.cvtColor(np.array(left), cv2.COLOR_RGB2GRAY)  
right = cv2.cvtColor(np.array(right), cv2.COLOR_RGB2GRAY)

# detect FAST corners in both images
fast = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
kp1 = fast.detect(left, None)
kp2 = fast.detect(right, None)

# Initiate BRIEF extractor
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

# compute the descriptors with BRIEF
kp1, desc1 = brief.compute(left, kp1)
kp2, desc2 = brief.compute(right, kp2)

# match the descriptors from both images
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(desc1, desc2)

# draw the first 100 matches
matches = sorted(matches, key = lambda x:x.distance)    # quickly sort the matches based on distance, smallest distance first
img = cv2.drawMatches(left,kp1,right,kp2,matches[:100], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(40,20))
plt.imshow(img)
plt.show()


## Try Matching Features between Consecutive Frames in Time

So far, we have matched keypoints between the left and right images of a stereo camera.
Let's try doing this between the left frames of the camera at different positions along the trajectory.


In [ ]:
# load the left images a few frames apart
left1, _ = data.get_rgb(10)
left2, _ = data.get_rgb(15)

left1 = cv2.cvtColor(np.array(left1), cv2.COLOR_RGB2GRAY)  
left2 = cv2.cvtColor(np.array(left2), cv2.COLOR_RGB2GRAY)  

# use plt to show them side by side
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(left1, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(left2, cmap='gray')
plt.show()

# detect and match ORB features
orb = cv2.ORB_create()               # Your turn! Explore different parameters you can pass into the SIFT constructor
kp1, desc1 = orb.detectAndCompute(left1, None)
kp2, desc2 = orb.detectAndCompute(left2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(desc1, desc2)
matches = sorted(matches, key = lambda x:x.distance)    # quickly sort the matches based on distance, smallest distance first

img = cv2.drawMatches(left1,kp1,left2,kp2,matches[:100], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(40,20))
plt.imshow(img)
plt.show()


# YOUR TURN!
# Try matching from frame 10 to frames increasingly further away (15, 20, 25, 50, ...)
# What do you notice? Is there a point where the matches start to get worse? Why do you think that is?
# How do the distances of the found matches behave? Do they increase? Decrease? Stay the same?
# Repeat this experiment with ORB and SIFT. Which one is more robust to changes in viewpoint?
